# Elemente des Modells

### Einrichtung

Die erste Zeile dient dazu, eine moeglichst hohe Kompatibilitaet zwischen Python 2 und 3 zu sichern

In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import io

import pypsa
import pypsa.plot as bmplot

%matplotlib inline

## Setup

Einrichtung aller wichtigen Kompnenten und einlesen der Daten

### Variablen

In [2]:
nw = pypsa.Network()

In [3]:
# dates = ['morgens', 'mittags', 'abends']

dates = pd.date_range("20170101", periods=6, freq="4H")

nw.set_snapshots(dates)

### Einlesen der Datenbanken

In [4]:
cur_path = r"C:\Users\janni\OneDrive\TU Berlin\4 - Abschlussarbeiten\01 - Bachelorarbeit\modell\data\\"

In [5]:
components = [pd.DataFrame for i in range(4)]
filenames = ["vertices", "lines", "unit_data_twogen","loads"]
units = ["Bus", "Line", "Generator", "Load"]

Einlesen der als *.csv gespeicherten Daten in ein pandas DataFrame, alle NaN werden durch 0 ersetzt

In [6]:
for i in range(len(components)):
    components[i] = pd.read_csv(cur_path + filenames[i] + ".csv")
    components[i].fillna(0,inplace=True)

In [7]:
components[1].rename(columns={"wid":"x","len":"length","cap":"s_nom"},inplace=True)
components[2].rename(columns={"marg. Cost":"marginal_cost","min power":"p_min_pu","startup costs":"start_up_cost","nom power":"p_nom"},inplace=True)

## From Excel workbooks

gen_data = [pd.DataFrame for i in range(2)]
ex = pd.ExcelFile(cur_path + "Technische Daten Gaskraftwerke.xlsx")

for i in range(len(gen_data)):
    # Extract last two columns from worksheet and only rows 5-12
    gen_data[i] = ex.parse(i).iloc[4:11,[-2,-1]]
    # Use values from first column as index
    gen_data[i].index = gen_data[i].iloc[:,0]
    # Delete column in the middle
    gen_data[i] = gen_data[i].iloc[:, -1]
    
gen_data

### Daten Last und erzeugte Windkraft

lst = pd.read_excel(cur_path + "Stromlast und Windeinspeisung.xlsx",
                    sheet_name="Tabelle1",
                    header=3,
                    index_col=0,
                    parse_cols="A:F")

# Extract only the first day, every Xh
intervall = 16
smaller = lst.iloc[intervall::intervall,[0,1]].head((96//intervall)-1)
smaller

Uebertragen der Daten von pandas in das Netzwerk

In [10]:
for i in range(len(components)):
    pypsa.io.import_components_from_dataframe(nw, components[i], units[i])

Einlesen der Werte pro Zeitintervall

In [18]:
nw.import_series_from_dataframe(pd.read_csv(cur_path + "load_dist2" + ".csv"),
                                "Load",
                                "p_set")

               '2017-01-01 08:00:00', '2017-01-01 12:00:00',
               '2017-01-01 16:00:00', '2017-01-01 20:00:00'],
              dtype='datetime64[ns]', freq=None) are missing from p_set of Load


KeyError: "None of [DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 04:00:00',\n               '2017-01-01 08:00:00', '2017-01-01 12:00:00',\n               '2017-01-01 16:00:00', '2017-01-01 20:00:00'],\n              dtype='datetime64[ns]', freq='4H')] are in the [index]"

In [12]:
# dates = pd.date_range("20170101", periods=6, freq="4H")

df = pd.DataFrame(pd.read_csv(cur_path + "load_dist2" + ".csv"))
df.index = dates
nw.import_series_from_dataframe(df, "Load", "p_set")

In [10]:
for i in range(len(components)):
    pypsa.io.import_components_from_dataframe(nw, components[i], units[i])

Einlesen der Werte pro Zeitintervall

In [18]:
nw.import_series_from_dataframe(pd.read_csv(cur_path + "load_dist2" + ".csv"),
                                "Load",
                                "p_set")

               '2017-01-01 08:00:00', '2017-01-01 12:00:00',
               '2017-01-01 16:00:00', '2017-01-01 20:00:00'],
              dtype='datetime64[ns]', freq=None) are missing from p_set of Load


KeyError: "None of [DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 04:00:00',\n               '2017-01-01 08:00:00', '2017-01-01 12:00:00',\n               '2017-01-01 16:00:00', '2017-01-01 20:00:00'],\n              dtype='datetime64[ns]', freq='4H')] are in the [index]"

In [12]:
# dates = pd.date_range("20170101", periods=6, freq="4H")

df = pd.DataFrame(pd.read_csv(cur_path + "load_dist2" + ".csv"))
df.index = dates
nw.import_series_from_dataframe(df, "Load", "p_set")

In [13]:
imp = pd.read_excel(cur_path + "Technische Daten Gaskraftwerke.xlsx",
                    sheet_name="GuD Kraftwerk",
                    header=None,
                    skiprows=5,
                    index_col=0,
                    index=False,
                    parse_cols="C,D",
                    skip_footer=28)
additional_gen_vals = { "p_min":50,
                      "marginal_cost":35}

for i,j in additional_gen_vals.items():
    imp.loc[i] = j
imp

,1
0,
name,Fortuna
p_nom,595
efficiency,0.6039
carrier,gas
ramp_limit_up,35
ramp_limit_down,35
p_min,50
marginal_cost,35


In [14]:
# imp.to_csv(header=None, index=False)
# Get rid of unnecessary index column header
tst = pd.read_csv(io.StringIO(u""+imp.to_csv(header=None,index_cols=1)), header=None)
tst

TypeError: to_csv() got an unexpected keyword argument 'index_cols'

Einlesen der Lastdaten & Windkraft

In [ ]:
lst = pd.read_excel(cur_path + "Stromlast und Windeinspeisung.xlsx",
                    sheet_name="Tabelle1",
                    header=3,
                    #skiprows=3,
                    index_col=0,
                    parse_cols="A:F")

## Optimierung

In [17]:
nw.generators

,bus,capital_cost,carrier,commitable,committable,control,efficiency,initial_status,marginal_cost,min_down_time,...,p_set,q_set,ramp_limit_down,ramp_limit_shut_down,ramp_limit_start_up,ramp_limit_up,shut_down_cost,sign,start_up_cost,type
0,3,0.0,wind,1.0,False,PQ,0.45,1,5.0,0,...,0.0,0.0,NaN,1.0,1.0,NaN,0.0,1.0,0.0,
1,1,0.0,gas,1.0,False,PQ,0.38,1,50.0,0,...,0.0,0.0,NaN,1.0,1.0,NaN,0.0,1.0,5000.0,
2,2,0.0,gas,1.0,False,PQ,0.60,1,35.0,0,...,0.0,0.0,NaN,1.0,1.0,NaN,0.0,1.0,6000.0,


## Aufbereitung der Daten

In [15]:
gen_outs = nw.generators_t.p
caps_by_carrier = nw.generators.groupby("carrier")["p_nom"].sum()
gen_pot_remaining = nw.generators.p_nom - gen_outs
print(gen_outs)
gen_outs.loc[dates[1],"1"]

Empty DataFrame
Columns: []
Index: [2017-01-01 00:00:00, 2017-01-01 04:00:00, 2017-01-01 08:00:00, 2017-01-01 12:00:00, 2017-01-01 16:00:00, 2017-01-01 20:00:00]


KeyError: 'the label [1] is not in the [index]'

In [ ]:
#dates = pd.date_range("2017-01-01", periods=6, freq="4H")
#dates

In [16]:
gen_pot_remaining

,0,1,2
2017-01-01 00:00:00,NaN,NaN,NaN
2017-01-01 04:00:00,NaN,NaN,NaN
2017-01-01 08:00:00,NaN,NaN,NaN
2017-01-01 12:00:00,NaN,NaN,NaN
2017-01-01 16:00:00,NaN,NaN,NaN
2017-01-01 20:00:00,NaN,NaN,NaN


## Visualization

In [ ]:
diag_rows = len(nw.generators)
diag_cols = len(nw.snapshots)
diag_inches = 4

fig, axes = plt.subplots(nrows=diag_rows, ncols=diag_cols)
fig.set_size_inches(diag_inches*diag_cols, diag_inches*diag_cols)

for i in range(diag_rows):
    for j in range(diag_cols):
        perc_labels = ["Available", "Dispatched"]
        sizes = [gen_pot_remaining.loc[dates[j],"{0}".format(i)],
                 gen_outs.loc[dates[j],"{0}".format(i)]]
        colors = ["green", "blue"]

        axes[i,j].pie(sizes, labels=perc_labels, colors=colors, autopct="%1.1f%%")
        axes[i,j].axis("equal")
        axes[0,j].set_title(dates[j])
        axes[i,0].set_ylabel(components[2]["name"][i])
        plt.subplots_adjust(wspace=0.7, hspace=0.2)
        #plt.pie(fig, labels=perc_labels, colors=colors, autopct="%1.1f%%")
        #plt.axis("equal")

Line graph to display the dispatched amount of energy per plant

In [ ]:
gen_plot = gen_outs.plot(kind="area",grid=True)
lines, labels = gen_plot.get_legend_handles_labels()
gen_plot.legend(lines[:len(nw.generators)], components[2]['name'].values.tolist(), loc='best')
plt.ylabel("MW")

network graph showing the distribution of buses etc.

In [ ]:
bmplot.plot(nw, title="Aufbau Netz")
plt.xlim([0,120])
plt.ylim([0,120])